In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

In [2]:

# --- 1. 학습용 데이터 (Corpus) 준비 ---

# 간단하고 반복적인 텍스트가 학습에 용이합니다.
corpus_text = (
    "Mary had a little lamb, its fleece was white as snow. "
    "And everywhere that Mary went, the lamb was sure to go. "
    "It followed her to school one day, which was against the rule. "
    "It made the children laugh and play, to see a lamb at school."
)

In [3]:

# 1.1. 전처리 (1단계와 유사하나, 불용어/어간추출 안 함)
# 언어 모델은 'a', 'the', 'was' 같은 단어도 예측해야 하므로 제거하지 않습니다.
tokens = corpus_text.lower().split()

# 1.2. 어휘 사전 (Vocabulary) 구축
# 각 고유 단어에 정수 인덱스(ID)를 부여합니다.
vocab = sorted(list(set(tokens)))
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}

VOCAB_SIZE = len(vocab)
print(f"총 어휘 수 (VOCAB_SIZE): {VOCAB_SIZE}")

총 어휘 수 (VOCAB_SIZE): 36


In [13]:


# 1.3. 학습 데이터셋 (Sliding Window) 생성
# (예: "Mary had a" -> "little")
CONTEXT_SIZE = 3 # 예측을 위해 참고할 이전 단어 수
sequences = []

# "슬라이딩 윈도우" 방식으로 (X, y) 페어를 만듭니다.
# [Image of sliding window for NLP text data]
for i in range(len(tokens) - CONTEXT_SIZE):
  context = tokens[i : i + CONTEXT_SIZE]
  target = tokens[i + CONTEXT_SIZE]

  # 단어를 인덱스로 변환
  context_indices = [word_to_idx[w] for w in context]
  target_index = word_to_idx[target]

  sequences.append((context_indices, target_index))

print(f"총 학습 시퀀스 수: {len(sequences)}")
print(f"첫 번째 시퀀스 (X, y): {sequences[0]}")
print(f"  -> ({[idx_to_word[i] for i in sequences[0][0]]}, {idx_to_word[sequences[0][1]]})")


총 학습 시퀀스 수: 44
첫 번째 시퀀스 (X, y): ([20, 11, 0], 18)
  -> (['mary', 'had', 'a'], little)


In [14]:
# --- 2. 헬퍼 함수 (데이터 -> 텐서) ---
def sequence_to_tensor(context_indices, target_index):
  # X (Context) 텐서
  context_tensor = torch.tensor(context_indices, dtype=torch.long)
  # y (Target) 텐서
  target_tensor = torch.tensor([target_index], dtype=torch.long)
  return context_tensor, target_tensor



In [18]:
# --- 3. LSTM 언어 모델 정의 (수정) ---

class WordLSTM_LM(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim):
    super(WordLSTM_LM, self).__init__()

    # [수정 1] hidden_dim을 self 변수로 저장합니다.
    self.hidden_dim = hidden_dim

    # 4단계 개념: (중요) 임베딩 레이어
    self.embedding = nn.Embedding(vocab_size, embedding_dim)

    # 3단계 개념: LSTM 레이어
    self.lstm = nn.LSTM(embedding_dim, hidden_dim)

    # 3단계 개념: 출력 레이어
    self.linear = nn.Linear(hidden_dim, vocab_size)

    # LogSoftmax (NLLLoss와 짝)
    self.log_softmax = nn.LogSoftmax(dim=1)

  def forward(self, context_tensor, hidden, cell):
    # context_tensor: [CONTEXT_SIZE]

    # 1. 임베딩: [CONTEXT_SIZE] -> [CONTEXT_SIZE, EMBEDDING_DIM]
    embeds = self.embedding(context_tensor).view(len(context_tensor), 1, -1)

    # 2. LSTM:
    lstm_out, (hidden, cell) = self.lstm(embeds, (hidden, cell))

    # 3. Linear: 시퀀스의 '마지막' 출력만 사용
    output = self.linear(lstm_out[-1])

    # 4. Softmax
    log_probs = self.log_softmax(output)

    return log_probs, hidden, cell

  def init_hidden_cell(self, batch_size=1):
    # [수정 2] hidden_dim 대신 self.hidden_dim을 사용합니다.
    return (torch.zeros(1, batch_size, self.hidden_dim),
            torch.zeros(1, batch_size, self.hidden_dim))

In [16]:
# --- 4. 모델 학습 ---

# 4.1. 하이퍼파라미터 설정
EMBEDDING_DIM = 50  # 4단계의 vector_size와 동일한 개념
HIDDEN_DIM = 64     # 3단계의 hidden_size와 동일한 개념
N_EPOCHS = 200
learning_rate = 0.01

In [19]:

model = WordLSTM_LM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM)
criterion = nn.NLLLoss() # LogSoftmax와 짝
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print("\n--- 모델 학습 시작 ---")

for epoch in range(1, N_EPOCHS + 1):
  epoch_loss = 0

  # (데이터셋이 작으므로 매번 셔플하며 학습)
  random.shuffle(sequences)

  for context_indices, target_index in sequences:
    # 1. 데이터 준비
    context_tensor, target_tensor = sequence_to_tensor(context_indices, target_index)

    # 2. 그래디언트/은닉 상태 초기화
    model.zero_grad()
    hidden, cell = model.init_hidden_cell()

    # 3. 순전파 (Forward)
    log_probs, hidden, cell = model(context_tensor, hidden, cell)

    # 4. 손실 계산 및 역전파
    loss = criterion(log_probs, target_tensor)
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

  if epoch % 20 == 0:
    print(f"Epoch {epoch:3d} / {N_EPOCHS} | Avg Loss: {epoch_loss / len(sequences):.4f}")

print("--- 모델 학습 완료 ---")


--- 모델 학습 시작 ---
Epoch  20 / 200 | Avg Loss: 0.0032
Epoch  40 / 200 | Avg Loss: 0.0009
Epoch  60 / 200 | Avg Loss: 0.0004
Epoch  80 / 200 | Avg Loss: 0.0002
Epoch 100 / 200 | Avg Loss: 0.0001
Epoch 120 / 200 | Avg Loss: 0.0001
Epoch 140 / 200 | Avg Loss: 0.0000
Epoch 160 / 200 | Avg Loss: 0.0000
Epoch 180 / 200 | Avg Loss: 0.0000
Epoch 200 / 200 | Avg Loss: 0.0000
--- 모델 학습 완료 ---


In [20]:

# --- 5. 모델 평가 (다음 단어 예측) ---

def predict_next_word(seed_text):
  print(f"\n--- 예측 테스트 ---")
  print(f"입력: '{seed_text}'")

  # (중요) 평가 모드 + 그래디언트 계산 중지
  model.eval()
  with torch.no_grad():
    # 1. 입력 텍스트 전처리
    seed_tokens = seed_text.lower().split()
    if len(seed_tokens) < CONTEXT_SIZE:
      print(f"Error: 입력 텍스트는 최소 {CONTEXT_SIZE}단어여야 합니다.")
      return

    # 마지막 CONTEXT_SIZE 만큼의 단어만 사용
    context_tokens = seed_tokens[-CONTEXT_SIZE:]
    context_indices = [word_to_idx[w] for w in context_tokens]
    context_tensor = torch.tensor(context_indices, dtype=torch.long)

    # 2. 모델 예측
    hidden, cell = model.init_hidden_cell()
    log_probs, _, _ = model(context_tensor, hidden, cell)

    # 3. 결과 해석
    # 가장 확률이 높은(log_prob가 가장 큰) 단어의 인덱스
    predicted_idx = torch.argmax(log_probs).item()
    predicted_word = idx_to_word[predicted_idx]

    print(f"예측된 다음 단어: '{predicted_word}'")

In [21]:


# 테스트 실행
predict_next_word("Mary had a")
predict_next_word("lamb was sure")
predict_next_word("children laugh and")


--- 예측 테스트 ---
입력: 'Mary had a'
예측된 다음 단어: 'little'

--- 예측 테스트 ---
입력: 'lamb was sure'
예측된 다음 단어: 'to'

--- 예측 테스트 ---
입력: 'children laugh and'
예측된 다음 단어: 'play,'
